In [1]:
import pyautogui as pag
import numpy as np
import mss, cv2, win32gui, win32ui, win32con, pdb, time
import matplotlib.pyplot as plt
import copy

In [370]:
pos = pag.position()
print(pos)

Point(x=250, y=835)


In [9]:
def check_num(color):
    global color_list
    color_list = np.array([[255, 246, 222], # 0 
                           [255, 254, 255], # 2
                           [255, 236, 170], # 4
                           [255, 229, 143], # 8
                           [255, 216,  87], # 16
                           [241, 183,   0], # 32
                           [204, 140,   1], # 64
                           [ 60, 240, 141], # 128
                           [ 32, 232, 109], # 256z
                           [ 26, 222,  77], # 512
                           [121, 225, 255], # 1024
                           [ 13, 201, 255], # 2048 
                           [  1, 151, 255]]) # 4096
    color = color.flatten()
    # loc = np.where((abs(color_list[:, 0] - color[0]) < 3) * \
    #                (abs(color_list[:, 1] - color[1]) < 3) * \
    #                (abs(color_list[:, 2] - color[2]) < 3))[0]
    
    loc = np.argmin(np.sum((color_list - color)**2, axis=1))
    return loc.item()

def get_tile_array(tile_xpos, tile_ypos):
    tile_array = np.zeros((4, 4))
    for ii in range(4):
        for jj in range(4):
            with mss.mss() as sct:
                color = np.array(sct.grab(
                    {'left':tile_xpos[ii], 'top':tile_ypos[jj], 
                     'width':1, 'height':1}))[:, :, :3]
                pass
            # print(color)
            tile_array[jj, ii] = check_num(color)
    return tile_array

def after_moving(tile_array, direction='up'):
    if direction == 'up':
        tile_array0 = copy.deepcopy(tile_array)
    if direction == 'down':
        tile_array0 = copy.deepcopy(tile_array[::-1])
    if direction == 'right':
        tile_array0 = copy.deepcopy(tile_array.T[::-1])
    if direction == 'left':
        tile_array0 = copy.deepcopy(tile_array.T)

    templet = np.zeros((4, 4))
    for i in range(4): # summing
        part = tile_array0[:, i]
        part0 = part[part > 0]
        for j in range(len(part0)-1):
            if part0[j] == part0[j+1]:
                part0[j] += 1
                part0[j+1] = 0
        part1 = part0[part0 > 0]
        templet[:len(part1), i] = part1

    if direction == 'down':
        templet = templet[::-1]
    if direction == 'right':
        templet = templet[::-1].T
    if direction == 'left':
        templet = templet.T
    return templet

def evaluation(after_tile_array, table):
    # print(table)
    dum = after_tile_array
    # dum = np.where(after_tile_array==0, 2, after_tile_array)
    return np.sum(table*(2**dum))

def find_direct(tile_array, level):
    score = 0
    direct = 'down'
  
    for try_dir in ['left', 'up', 'right']:
        after = after_moving(tile_array, direction=try_dir)
        if (after == tile_array).all(): 
            print('a')
            continue
        else:
            # if tile_array[0, 0] == 0:
            #     return try_dir, 10 
            if level <= 2:
                direct_sub, score_sub = find_direct(after, level+1)
            else:
                new_score = evaluation(after, table)
                return try_dir, new_score
        if (score < score_sub):# and (direct_sub != 'down'):
            score = score_sub
            direct = try_dir
    return direct, score

def drag(direction, delay=1e-3):
    pag.moveTo(220, 779)
    delta = 30
    if direction=='up': pag.drag(0, -delta, delay, button='left')
    if direction=='down': pag.drag(0, delta, delay, button='left')    
    if direction=='right': pag.drag(delta, 0, delay, button='left')
    if direction=='left': pag.drag(-delta, 0, delay, button='left')

In [3]:
challenge_mode_pos = [220, 719]
practice_mode_pos = [220, 779]
tile_xpos = np.arange(4)*100 + 50
tile_ypos = np.arange(4, dtype=int)*100 + 550

global table
table0 = np.arange(4*4)[::-1].reshape(4, 4)
table = 2.**(table0)
# table = table0**2
table[1] = table[1][::-1]
table[3] = table[3][::-1]
# table[0, 0] = table[0, 0]*1e4
# table = table * 10**np.arange(4)[::-1, None]

# table = np.array([[10,8,7,6.5],
#  [.5,.7,1,3],
#  [-.5,-1.5,-1.8,-2],
#  [-3.8,-3.7,-3.5,-3]])

aa = 2**np.arange(4)[::-1]
table = aa*aa[:, None]

In [5]:
mode = 'challenge'
mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(2)
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, 0)
    drag(final_dir)
    # if (final_dir == 'right') and \
    #     ((after_moving(tile_array, 'right'))[0, 0] == 0):
    #     drag('left')
    time.sleep(0.2)

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [110]:
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, 0)
    print("\r", final_dir, score, end="")
    # time.sleep(0)

 up 1266430.02.0

KeyboardInterrupt: 

In [10]:
tile_array = np.array([[3, 2, 2, 4], 
                       [1, 2, 2, 0], 
                       [0, 0, 0, 0], 
                       [1, 1, 0, 0]])
tile_array = get_tile_array(tile_xpos, tile_ypos)
final_dir, score = find_direct(tile_array, 0)   
final_dir, score

a
a
a


('down', 0)

In [49]:
evaluation(after_moving(tile_array, 'up'), table)

np.float64(532990.0)

In [50]:
evaluation(after_moving(tile_array, 'left'), table)

np.float64(543744.0)

In [61]:
get_tile_array(tile_xpos, tile_ypos)

array([[5., 7., 6., 5.],
       [2., 5., 3., 6.],
       [1., 3., 1., 2.],
       [0., 0., 0., 0.]])

In [62]:
final_dir

'up'

In [64]:
for try_dir in ['up', 'down', 'left', 'right']:
    after = after_moving(tile_array, direction=try_dir)
    if (after == tile_array).all(): continue
    new_score = evaluation(after, table)
    print(try_dir, new_score)
    if score < new_score: 
        score = new_score
        final_dir = try_dir

up 367.0
down -198.0
left 97.0
right 72.0


In [39]:
mode = 'challenge'
mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(2)
n = 0
tile_array = np.zeros((4, 4))
while True:
    # try:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir = None
    final_dir, score = find_direct(tile_array, 0)
    drag(final_dir)

right

press 


down

press q


up

press ased


up

KeyboardInterrupt: Interrupted by user

In [93]:
mode = 'challenge'
mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(2)
n = 0
tile_array = np.zeros((4, 4))
while True:
    # try:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir = None
    score = 0
    # for try_dir in ['up', 'down', 'left', 'right']:
    for try_dir in ['up', 'left', 'right']:
        after = after_moving(tile_array, direction=try_dir)
        if (after == tile_array).all(): continue
        new_score = evaluation(after, table)
        if score < new_score: 
            score = new_score
            final_dir = try_dir
    if final_dir == None: final_dir = 'down'
    drag(final_dir)
    # print(n, final_dir)
    # time.sleep(0.1)
# except:
    #     print(tile_array)
    #     print(final_dir)
    # break

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.